In [0]:
%sql
-- NOTEBOOK: 03_Data_Quality_Checks

USE pl_delivery_analysis;

-- ===========================================================================
-- POR QUE FAZEMOS ISSO? 
-- Evitar o "Garbage In, Garbage Out". Se entra lixo, sai lixo.
-- Em produção, dados ruins causam decisões de negócio erradas.
-- ===========================================================================

-- 1. CHECKS DA CAMADA BRONZE (Dados brutos)
CREATE OR REPLACE TEMPORARY VIEW bronze_data_quality AS
SELECT 
    'orders_bronze' as tabela,
    COUNT(*) as total_registros,
    
    -- POR QUE VERIFICAR CHAVES ÚNICAS?
    -- Duplicatas podem inflar métricas e causar cálculos errados
    COUNT(DISTINCT order_id) as chaves_unicas,
    
    -- POR QUE VERIFICAR NULOS EM CHAVES?
    -- Chaves nulas quebram JOINs e causam perda de dados
    SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) as nulos_chave_primaria,
    
    -- POR QUE VERIFICAR VALORES NEGATIVOS?
    -- Valores financeiros negativos não fazem sentido neste contexto
    SUM(CASE WHEN CAST(subtotal AS DECIMAL(10,2)) < 0 THEN 1 ELSE 0 END) as subtotal_negativo,
    SUM(CASE WHEN CAST(delivery_fee AS DECIMAL(10,2)) < 0 THEN 1 ELSE 0 END) as delivery_fee_negativo,
    
    -- CALCULAR % DE PROBLEMAS (importante para monitoramento)
    ROUND(
        (SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*)),
    2) as percentual_nulos

FROM tbl_fact_orders_bronze;

-- 2. CHECKS DA CAMADA SILVER (Dados transformados)
CREATE OR REPLACE TEMPORARY VIEW silver_data_quality AS
SELECT 
    'pedidos_silver' as tabela,
    COUNT(*) as total_registros,
    
    -- POR QUE VERIFICAR PEDIDOS COM PREJUÍZO?
    -- Muitos pedidos negativos podem indicar problema no modelo de negócio
    SUM(CASE WHEN lucro_bruto_unitario < 0 THEN 1 ELSE 0 END) as pedidos_prejuizo,
    
    -- POR QUE CALCULAR MARGEM MÉDIA?
    -- Para entender a saúde geral do negócio
    AVG(lucro_bruto_unitario) as margem_media,
    
    -- POR QUE VERIFICAR GMV INVÁLIDO?
    -- GMV <= 0 indica problemas graves nos dados de entrada
    SUM(CASE WHEN gmv_total <= 0 THEN 1 ELSE 0 END) as gmv_invalido,

    -- CALCULAR % DE PEDIDOS COM PROBLEMAS
    ROUND(
        (SUM(CASE WHEN lucro_bruto_unitario < 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)),
    2) as percentual_prejuizo

FROM tbl_fact_pedidos_silver;

-- 3. RELATÓRIO FINAL DE QUALIDADE
SELECT 
    tabela,
    total_registros,
    chaves_unicas,
    nulos_chave_primaria,
    percentual_nulos,
    
    -- LÓGICA DE ALERTAS: Definir limites aceitáveis
    CASE 
        WHEN nulos_chave_primaria > 0 THEN '🔴 CRÍTICO: Chaves primárias nulas'
        WHEN chaves_unicas != total_registros THEN '🟡 ALERTA: Possíveis duplicatas'
        WHEN percentual_nulos > 5 THEN '🟡 ALERTA: Muitos nulos'
        ELSE '✅ OK'
    END as status_qualidade
    
FROM bronze_data_quality

UNION ALL

SELECT 
    tabela,
    total_registros,
    NULL as chaves_unicas,  -- Não aplicável aqui
    pedidos_prejuizo as nulos_chave_primaria, -- Reutilizando coluna para outro alerta
    percentual_prejuizo as percentual_nulos,
    
    CASE 
        WHEN percentual_prejuizo > 20 THEN '🔴 CRÍTICO: Muitos pedidos com prejuízo'
        WHEN percentual_prejuizo > 10 THEN '🟡 ALERTA: Percentual de prejuízo alto'
        ELSE '✅ OK'
    END as status_qualidade
    
FROM silver_data_quality;